# dina-py Library Tutorial

This notebook demonstrates how to use the dina-py Python library for interacting with DINA.

# 🛠 Installation Guide

This notebook walks you through setting up your environment and installing the `dinapy` library for use in Jupyter Notebooks.

# Manual Setup (outside the notebook)

## ✅ Step 1: Make sure to have Python 3.10 or newer

## ✅ Step 2: Create a Virtual Environment

Run the following command in your TERMINAL (NOT Jupyter notebook):

In [ ]:
python -m venv env


### 🔹 Activate the Environment

- **Windows:**


In [ ]:
.\env\Scripts\activate

- **macOS/Linux:**

In [ ]:
source env/bin/activate

## ✅ Step 3: Install `dinapy`

### From Git (branch) — works in Jupyter or terminal
If on Jupyter notebook, just run the cell below:

In [ ]:
# Start clean: remove any previously installed dinapy
%pip uninstall -y dinapy

# Equivalent to your original, but more resilient parsing for extras:
%pip install "dinapy[notebook] @ git+https://github.com/AAFC-BICoE/dina-py.git@Support_36999_jupyter_notebook_test"

## ✅ Step 4: Launch Jupyter Notebook

You can now launch Jupyter Notebook server with the following command in your TERMINAL:

In [ ]:
jupyter notebook

## Import the Library

Let's import the dina-py library here and check if it's working.

In [8]:
# Import dina-py
from importlib.metadata import version, PackageNotFoundError

try:
    import dinapy
    print(f"dina-py successfully imported!")

    try:
        print("Version:", version("dinapy"))
    except PackageNotFoundError:
        print("Version: Unknown (package not installed)")
except ImportError as e:
    print(f"Error importing dina-py: {e}")
    print("Please install it using: pip install dina-py")

dina-py successfully imported!
Version: 0.1.0


## Configuration

Check DINA connection credentials from Environment variables.

In [ ]:
# Configuration variables checks
DINA_BASE_URL = "https://your-dina-instance.org"  # Replace with your DINA instance URL

print("Configuration set. Ready to connect to DINA.")

## Initialize DINA Client

Create a DINA client instance to interact with the API.

In [ ]:
# Initialize the DINA client
try:
    client = dina.DinaClient(
        base_url=DINA_BASE_URL,
        username=USERNAME,
        password=PASSWORD
    )
    print("DINA client initialized successfully!")
except Exception as e:
    print(f"Error initializing DINA client: {e}")

## Example: Query Data

Let's try to query some data from DINA.

In [ ]:
# Example: Get a list of specimens
try:
    specimens = client.get_specimens(limit=10)
    print(f"Retrieved {len(specimens)} specimens:")
    for specimen in specimens:
        print(f"- {specimen}")
except Exception as e:
    print(f"Error retrieving specimens: {e}")

## Example: Search Functionality

Perform a search query on DINA data.

In [ ]:
# Example: Search for specific records
try:
    search_term = "example"  # Replace with your search term
    results = client.search(query=search_term)
    print(f"Search results for '{search_term}':")
    for result in results:
        print(f"- {result}")
except Exception as e:
    print(f"Error during search: {e}")

## Example: Create a New Record

Create a new record in DINA (adjust according to your data model).

In [ ]:
# Example: Create a new record
try:
    new_record = {
        "type": "specimen",
        "attributes": {
            "catalogNumber": "SPEC-12345",
            "scientificName": "Example species"
        }
    }
    
    created_record = client.create_record(new_record)
    print(f"Record created successfully: {created_record}")
except Exception as e:
    print(f"Error creating record: {e}")

## Example: Update a Record

Update an existing record in DINA.

In [ ]:
# Example: Update a record
try:
    record_id = "your-record-id"  # Replace with actual record ID
    updated_data = {
        "attributes": {
            "notes": "Updated notes field"
        }
    }
    
    updated_record = client.update_record(record_id, updated_data)
    print(f"Record updated successfully: {updated_record}")
except Exception as e:
    print(f"Error updating record: {e}")

## Helper Functions

Here are some helper functions that might be useful when working with dina-py.

In [ ]:
import pandas as pd
import json

def display_records_as_dataframe(records):
    """Convert DINA records to a pandas DataFrame for easier viewing"""
    if not records:
        print("No records to display")
        return None
    
    df = pd.DataFrame(records)
    return df

def pretty_print_json(data):
    """Pretty print JSON data"""
    print(json.dumps(data, indent=2))

print("Helper functions loaded!")

## Your Custom Code

Use the cells below to write your own code using dina-py.

In [ ]:
# Your code here


## Resources

- [DINA Documentation](https://dina-project.github.io/)
- [dina-py GitHub Repository](https://github.com/DINA-Web/dina-py)
- [DINA API Documentation](https://api.dina-project.net/)

## Notes

- Make sure to replace placeholder values with your actual DINA instance credentials
- The exact methods and parameters may vary depending on the version of dina-py you're using
- Always refer to the official documentation for the most up-to-date information
- Be careful with credentials - consider using environment variables for sensitive information